# Configure

In [100]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [101]:
from zipfile import ZipFile
import zipfile_deflate64
import os
from pathlib import Path
import sys
import glob
import shutil
from os.path import join

os.environ["USE_PYGEOS"] = "0"
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rasterio.mask
from pyproj import CRS
import matplotlib.pyplot as plt

In [102]:
# Filepath directories

# Get the absolute path to the project directory
# Which is one directory above notebooks/
ABS_DIR = os.path.abspath(Path(os.getcwd()).parents[0])
# Get raw data directory
FR = join(ABS_DIR, 'data', 'raw')
# Get interim data directory
FI = join(ABS_DIR, 'data', 'interim')
# Get processed data directory
FP = join(ABS_DIR, 'data', 'processed')

# Directories for raw exposure, vulnerability (vuln) and 
# administrative reference files
#  all exist so just need references
EXP_DIR_R = join(FR, 'exposure')
VULN_DIR_R = join(FR, 'vuln')
REF_DIR_R = join(FR, 'ref')
# Haz is for FEMA NFHL and depth grids
HAZ_DIR_R = join(FR, 'haz')

# Directories for interim exposure, vulnerability (vuln) and 
# hazard
EXP_DIR_I = join(FI, 'exposure')
VULN_DIR_I = join(FI, 'vuln')
HAZ_DIR_I = join(FI, 'haz')
REF_DIR_I = join(FI, 'ref')

# Ensure they exist
Path(EXP_DIR_I).mkdir(parents=True, exist_ok=True)
Path(VULN_DIR_I).mkdir(parents=True, exist_ok=True)
Path(HAZ_DIR_I).mkdir(parents=True, exist_ok=True)
Path(REF_DIR_I).mkdir(parents=True, exist_ok=True)

# Reference fips
FIPS = '42101'

# Unzip Files

In [103]:
# For each .zip directory in fr
# Create needed subdirectories in interim/
# Unzip in the appropriate interim/ subdirectory

for path in Path(FR).rglob("*.zip"):
    # Avoid hidden files and files in directories
    if path.name[0] != ".":
        # Get root for the directory this .zip file is in
        zip_root = path.relative_to(FR).parents[0]

        # Get path to interim/zip_root
        zip_to_path = join(FI, zip_root)

        # Make directory, including parents
        # No need to check if directory exists bc
        # it is only created when this script is run
        Path(zip_to_path).mkdir(parents=True, exist_ok=True)

        # Unzip to zip_to_path
        with ZipFile(path, "r") as zip_ref:
            zip_ref.extractall(zip_to_path)

        print('Unzipped and moved to interim : '
              + str(path.name).split('.')[0])

Unzipped and moved to interim : noaa
Unzipped and moved to interim : tract
Unzipped and moved to interim : block
Unzipped and moved to interim : bg
Unzipped and moved to interim : zcta
Unzipped and moved to interim : county
Unzipped and moved to interim : nfhl
Unzipped and moved to interim : dg


# Process NSI

In [104]:
# The NSI comes with all the data necessary for performing a standard 
# flood risk assessment. It is still useful to process the raw data.
# Here, we subset to residential properties with 1 to 2 stories
# and save as a geodataframe. These are the types of residences we have
# multiple depth-damage functions for and a literature base to draw 
# from to introduce uncertainty in these loss estimates

In [105]:
# Read raw NSI data
nsi_filep = join(EXP_DIR_R, 'nsi.pqt')
# Read and reset index
nsi_full = pd.read_parquet(nsi_filep).reset_index(drop=True)

# Convert to geodataframe
geometry = gpd.points_from_xy(nsi_full['properties.x'],
                             nsi_full['properties.y'])
# The NSI CRS is EPSG 4326
nsi_gdf = gpd.GeoDataFrame(nsi_full, geometry=geometry,
                           crs="EPSG:4326")

# Drop the following columns
drop_cols = ['type', 'geometry.type', 'geometry.coordinates']
nsi_gdf = nsi_gdf.drop(columns=drop_cols)

# Remove "properties" from columns
col_updates = [x.replace("properties.", "") for x in nsi_gdf.columns]
nsi_gdf.columns = col_updates

In [106]:
# Subset to residential properties and update
# RES 1 - single family
# RES 2 - manufactured home
# RES 3 - multifamily (but could fit into a depth-damage function
# archetype depending on # stories)
# We are going to use RES1 for this case-study
# It is the only occtype with hazus and naccs
# DDFs and has less ambiguous classification

# occtype category for easier use in loss estimation steps

# Get residential structures
nsi_res = nsi_gdf.loc[nsi_gdf['occtype'].str[:4] == 'RES1']

# For this case-study, don't use any building with more 
# than 2 stories
res1_3s_ind = nsi_res['num_story'] > 2
# Final residential dataframe
res_f = nsi_res.loc[~res1_3s_ind]

In [107]:
# Subset to relevant columns
cols = ['fd_id', 'occtype', 'found_type', 'cbfips',
        'ftprntsrc', 'found_ht', 'val_struct',
        'val_cont', 'source', 'firmzone', 'ground_elv_m',
        'geometry']

res_out = res_f.loc[:,cols]

# Write out to interim/exposure/
EXP_OUT_FILEP = join(EXP_DIR_I, 'nsi_res.gpkg')
nsi_res.to_file(EXP_OUT_FILEP, driver='GPKG')

# Prepare depth-damage functions

In [108]:
# Read raw naccs data
# vuln/physical is a directory w/ files that 
# is pre-supplied for the user of this codebase
# The NACCS data is extracted from a pdf w/ manual entry
# I entered it as a dataframe that mimics the way
# Hazus enters data on DDFs so that it could potentially be 
# ingested into the Hazus database more easily
naccs = pd.read_csv(join(VULN_DIR_R, 'physical', 'naccs_ddfs.csv'))


In [109]:
# Goal of processing is to have the data in
# tidy format and with non string values

# I think we should stick to RES1. Change
# NSI processing above as well
# Need to change occ type codes for pile 
# foundation

# Drop Description and Source columns
# Melt on occupancy damage category
# Each depth is associated with a percent damage
dropcols = ['Description', 'Source']
idvars = ['Occupancy', 'DamageCategory']
naccs_melt = naccs.drop(columns=dropcols).melt(id_vars=idvars,
                                               var_name='depth_str',
                                               value_name='pct_dam')

# Need to convert depth_ft into a number
# Replace ft with empty character
# If string ends with m, make negative number
# Else, make positive number
naccs_melt['depth_str'] = naccs_melt['depth_str'].str.replace('ft', '')
negdepth = naccs_melt.loc[naccs_melt['depth_str'].str[-1] == 
                          'm']['depth_str'].str[:-1].astype(float)*-1
posdepth = naccs_melt.loc[naccs_melt['depth_str'].str[-1] != 
                          'm']['depth_str'].astype(float)

naccs_melt.loc[naccs_melt['depth_str'].str[-1] == 'm',
               'depth_ft'] = negdepth
naccs_melt.loc[naccs_melt['depth_str'].str[-1] != 'm',
               'depth_ft'] = posdepth

# Divide pctdam by 100
naccs_melt['rel_dam'] = naccs_melt['pct_dam']/100

# Delete depth_str and pctdam and standardize
# column names
dropcols = ['depth_str', 'pct_dam']
newcols = ['occtype', 'dam_cat', 'depth_ft', 'rel_dam']
naccs_melt = naccs_melt.drop(columns=dropcols)
naccs_melt.columns = newcols

# Write out to processed/vulnerability/
vuln_out_dir = join(VULN_DIR_I, 'physical')
Path(vuln_out_dir).mkdir(parents=True, exist_ok=True)
vuln_out_filep = join(vuln_out_dir, 'naccs_ddfs.csv')
naccs_melt.to_csv(vuln_out_filep, index=False)


In [110]:
# Got HAZUS DDFs from here: 
# https://github.com/cran/hazus/tree/master/data
# Downloaded the Hazus 5.1 technical manual and created a 
# Riverine IDs spreadsheet that tracks what the current version of 
# HAZUS recommends for riverine DDFs. Will cross reference that with 
# the downloaded DDFs from the cran/hazus/data/ repository. 
# I loaded the data in R (it’s .rda) and then converted to csv. 

# These are ddfs in the same form as the naccs data (I made the
# naccs data conform to this as best as I could)
hazus_ddfs = pd.read_csv(join(VULN_DIR_R, 'physical', 'haz_fl_dept.csv'))

In [111]:
# For basements, use FIA (MOD.) which does one and two floors by
# A and V zones
# For no basements, use USACE - IWR
# which does one and two floor, no flood zone specified
# 106: FIA (MOD.) 1S WB A zone
# 114: "" V zone
# 108: FIA (MOD.) 1S WB A zone
# 116: "" V zone

# 129: USACE - IWR 1S NB
# 130: USCAE - IWR 2S+ NB

# Handling pile and pier foundations is important
# for RES1, but this will not be an issue for this case-study
# since there are no foundation types like this in the NSI
# for Philly (is that true, though?)

# Subset to DmgFnId in the codes above
dmg_ids = [106, 108, 114, 116, 129, 130]
hazus_res = hazus_ddfs[(hazus_ddfs['DmgFnId'].isin(dmg_ids)) &
                       (hazus_ddfs['Occupancy'] == 'RES1')]

# Make occtype column in the same form that the NSI has
# e.g. RES1-1SNB
# Add column for A or V zone
# Note: outside SFHA basement homes will take A zone
# What other option do we have? 

# Split Description by comma. 
# The split[0] element tells us stories (but description sometimes
# says floors instead of story...)
# Can get around this issue by looking at first word
# The split[1] element
# tells us w/ basement or no basement. Use this to create occtype
desc = hazus_res['Description'].str.split(',')
s_type = desc.str[0].str.split(' ').str[0]
s_type = s_type.str.replace('one', '1').str.replace('two', '2')
b_type = desc.str[1].str.strip()
occtype = np.where(b_type == 'w/ basement',
                   s_type + 'SWB',
                   s_type + 'SNB')
fz = desc.str[-1].str.replace('Structure', '').str.strip()

# Need occtype, flood zone, depth_ft, and rel_dam columns
# Follow steps from naccs processing to get depth_ft and rel_dam
# First, drop unecessary columns
# Don't need Source_Table, Occupy_Class, Cover_Class, empty columns
# Description, Source, DmgFnId, Occupancy and first col (Unnamed: 0)
# because index was written out
# Don't need all na columns either (just for automobiles, apparently)
hazus_res = hazus_res.loc[:,[col for col in hazus_res.columns if 'ft' in col]]
hazus_res = hazus_res.dropna(axis=1, how='all')
# Add the occtype and fld_zone columns
hazus_res = hazus_res.assign(occtype=occtype,
                             fld_zone=fz.str[0])

# Then, occtype and fld_zone as index and melt rest of columns. Following 
# naccs processing
idvars = ['occtype', 'fld_zone']
hazus_melt = hazus_res.melt(id_vars=idvars,
                            var_name='depth_str',
                            value_name='pct_dam')

# Need to convert depth_ft into a number
# Replace ft with empty character
# If string ends with m, make negative number
# Else, make positive number
hazus_melt['depth_str'] = hazus_melt['depth_str'].str.replace('ft', '')
negdepth = hazus_melt.loc[hazus_melt['depth_str'].str[-1] == 
                          'm']['depth_str'].str[:-1].astype(float)*-1
posdepth = hazus_melt.loc[hazus_melt['depth_str'].str[-1] != 
                          'm']['depth_str'].astype(float)

hazus_melt.loc[hazus_melt['depth_str'].str[-1] == 'm',
               'depth_ft'] = negdepth
hazus_melt.loc[hazus_melt['depth_str'].str[-1] != 'm',
               'depth_ft'] = posdepth

# Divide pctdam by 100
hazus_melt['rel_dam'] = hazus_melt['pct_dam']/100

# Delete depth_str and pctdam and standardize
# column names
dropcols = ['depth_str', 'pct_dam']
newcols = ['occtype', 'fld_zone', 'depth_ft', 'rel_dam']
hazus_melt = hazus_melt.drop(columns=dropcols)
hazus_melt.columns = newcols

# Write out to processed/vulnerability/
vuln_out_dir = join(VULN_DIR_I, 'physical')
Path(vuln_out_dir).mkdir(parents=True, exist_ok=True)
vuln_out_filep = join(vuln_out_dir, 'hazus_ddfs.csv')
hazus_melt.to_csv(vuln_out_filep, index=False)

# Process Hazard

In [112]:
# Save the flood zones, do some processing on columns
# The files are unzipped as shape files instead of gdb
# We want S_FLD_HAZ_AR 
fld_haz_fp = join(HAZ_DIR_I, 'nfhl', 'S_FLD_HAZ_AR.shp')
nfhl = gpd.read_file(fld_haz_fp)

In [113]:
# Keep FLD_ZONE, FLD_AR_ID, STATIC_BFE, geometry
keep_cols = ['FLD_ZONE', 'FLD_AR_ID', 'STATIC_BFE', 'ZONE_SUBTY',
             'geometry']
nfhl_f = nfhl.loc[:,keep_cols]

# Adjust .2 pct X zones to X_500
nfhl_f.loc[nfhl_f['ZONE_SUBTY'] == '.2 PCT ANNUAL CHANCE FLOOD HAZARD',
           'FLD_ZONE'] = nfhl_f['FLD_ZONE'] + '_500'

# Update column names
# Lower case
nfhl_f.columns = [x.lower() for x in nfhl_f.columns]

# Drop ZONE_SUBTY
nfhl_f = nfhl_f.drop(columns=['zone_subty'])

# Write file
fz_out_dir = join(HAZ_DIR_I, 'fld_zon')
Path(fz_out_dir).mkdir(parents=True, exist_ok=True)
nfhl_f.to_file(join(fz_out_dir, 'floodzones.gpkg'),
               driver='GPKG')


In [114]:
# This is optional: delete the nfhl directory to reduce
# the file storage burden
# TODO: Make this a setting in a config file you can toggle as a user
RM_NFHL = True
if RM_NFHL:
    # Get directory name
    nfhl_dir = join(HAZ_DIR_I, 'nfhl')
    
    # Try to remove the tree; if it fails,
    # throw an error using try...except.
    try:
        shutil.rmtree(nfhl_dir)
    except OSError as e:
        print("Error: %s - %s." % (e.filename, e.strerror))
    

In [115]:
# No need to do anything for the depth rasters. Can 
# remove some extemperaneous files to reduce 
# size of project
# This is optional processing - make that clear
# In the interim/haz/dg directory, 
# only keep files that start with Depth_
# Can loop through files in the directory and keep all files
# that start with these characters
dg_dir = join(HAZ_DIR_I, 'dg')
for path in Path(dg_dir).iterdir():
    if str(path.name[:5]) != 'Depth':
        path.unlink()

# Process Reference Data

In [116]:
# Get all the reference files oriented to the county
# Save as .gpkg
# Clear ref/ directory when done

# base file name for state files
base_state_fp = 'tl_2022_42_'

# base file name for us files
base_us_fp = 'tl_2022_us_'

# state based files
state_ref_l = ['bg.shp', 'tabblock20.shp', 'tract.shp']
state_ref_l = [base_state_fp + x for x in state_ref_l]

# us based files
us_ref_l = ['zcta520.shp']
us_ref_l = [base_us_fp + x for x in us_ref_l]

# merge list
ref_l = state_ref_l + us_ref_l

In [117]:
# Read in county file
counties = gpd.read_file(join(REF_DIR_I, base_us_fp + 'county.shp'))

# Identify county from geoid column
# If processing multiple counties in the future, 
# can change to .isin(FIPS) w/ FIPS as a list
# Or, more generally can write a wrapper function
# for handling string or list input
counties_f = counties.loc[counties['GEOID'] == FIPS][['geometry']]
counties_f['fips'] = FIPS

# Use as reference for clipping other files
# For each polygon in counties_f, which corresponds to a county,
# you want to check temp_ref.within(polygon) and add these ref 
# polygons to a dataframe 


# Need a dict for these
# Start with counties_f which we need to write out
ref_clip_l = {'counties': counties_f}

# Loop through other data, clip to county, save as .gpkg
for ref in ref_l:
    # Read in the ref file
    temp_ref = gpd.read_file(join(REF_DIR_I, ref))
    # Do a spatial join for ref in county(ies)
    temp_ref_j = gpd.sjoin(temp_ref, counties_f, predicate='within')
    # Add to ref clip list
    # key/value pairs are refname.gpkg which can be obtained
    # by splitting refname.shp on '.' and keeping first part of string
    # then getting the last name after splitting on '_'
    ref_name = ref.split('.')[0].split('_')[-1]
    ref_clip_l[ref_name] = temp_ref_j
    # Helpful log message
    print('Clipped reference data: ' + ref_name)

# Delete the shp files in interim/ref
for path in Path(REF_DIR_I).iterdir():
    path.unlink()

# Save the gpkg files for each ref
# I could probably write out the gpkg into 
# a different directory, or something else
# that makes it obvious to just write the
# file out when you loop through ref_l above
# But reference files don't seem like
# processed data -- interim seems right
# An improvement might be having a temp directory
# for storing unzipped files and then
# it makes sense to clean these up
# after processing and moving certain files
# to interim. That seems neater and more
# nominally correct
for ref_name, ref in ref_clip_l.items():
    ref_out_fp = join(REF_DIR_I, ref_name + '.gpkg')
    # Wrote out ref data as gpkg
    ref.to_file(ref_out_fp, driver='GPKG')

Clipped reference data: bg
Clipped reference data: tabblock20
Clipped reference data: tract
Clipped reference data: zcta520
